In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os, sys, json
import numpy as np
import pandas as pd
#add parent dir to module search path to find readQueryCursor module
sys.path.insert(0,os.path.dirname(os.getcwd()))
#print(sys.path)
import module.readQueryCursor as rqc
import module.readStatistics as rsh
import module.readLinkMaintenance as rlm

dbname = 'testdb'
outputFormat = 'csv'
conf = rqc.readConnConfig.copy()
conf['database'] = dbname
conf['user'] = os.getenv('POSTGRES_USER')
conf['password'] = os.getenv('POSTGRES_PASSWORD')
myRQC = rqc.ReadQueryCursor(conf)

# set default connection for ReadStatsHelper
rsh.setReadStatsConnParameter(key='database', value=dbname)
rsh.setReadStatsConnParameter(key='user', value=conf['user'])
rsh.setReadStatsConnParameter(key='password', value=conf['password'])
myRSH = rsh.ReadStatisticsHelper(myRQC)



In [ ]:
def deleteEntities(tableName = None, prefix = None, entIDs = None):
  '''
  Mark for delete all entities tableName table with an prefix_id in entIDs
  '''
  if not isinstance(entIDs, np.ndarray) or len(entIDs) == 0 or tableName == None or prefix == None:
    print(f"aborting delete of {tableName} with ids {entIDs}")
    return None
  entCnt = len(entIDs)
  owneridColLabel = prefix + "_owner_id"
  visIDsColLabel = prefix + "_visibility_ids"
  pkColLabel = prefix + "_id"
  updCnt = 0
  for id in entIDs:
    myRQC.update(tableName,{owneridColLabel:"1",visIDsColLabel : "{" + "5}"},{pkColLabel:str(id)})
    #print("update query = ",myRQC.getQuery())
    if myRQC.getError():
      print(f"query to mark {tableName} entity id({id}) for delete failed", myRQC.getError())
      return myRQC.getError()
    else:
      updCnt += 1
  if updCnt != entCnt:
    return f"result count {updCnt} did not equal count requested {entCnt}"
  return f"Success: {entCnt} {tableName}s marked for delete"


def deleteTexts(txtIDs = None):
  return deleteEntities('text','txt',txtIDs)

def deleteText(txtID):
  return deleteTexts([txtID])


def deleteTextMetadatas(tmdIDs = None):
  return deleteEntities('textmetadata','tmd',tmdIDs)

def deleteTextMetadata(tmdID):
  return deleteTextMetadatas([tmdID])


def deleteEditions(ednIDs = None):
  return deleteEntities('edition','edn',ednIDs)

def deleteEdition(ednID):
  return deleteEditions([ednID])


def deleteSequences(seqIDs = None):
  return deleteEntities('sequence','seq',seqIDs)

def deleteSequence(seqID):
  return deleteSequences([seqID])


def deleteSyllableClusters(sclIDs = None):
  return deleteEntities('syllablecluster','scl',sclIDs)

def deleteSyllableCluster(sclID):
  return deleteSyllableClusters([sclID])


def deleteSegments(segIDs = None):
  return deleteEntities('segment','seg',segIDs)

def deleteSegment(segID):
  return deleteSegments([segID])


def deleteTokens(tokIDs = None):
  return deleteEntities('token','tok',tokIDs)

def deleteToken(tokID):
  return deleteTokens([tokID])


In [ ]:
'''
Find active texts that have 'to delete' in title or invNo or no title
'''
queryFindTextToDelete = \
" SELECT txt_id, txt_owner_id, edn_owner_id, txt_title, txt_ckn, edn_id, edn_description \
  FROM text \
    LEFT JOIN edition ON edn_text_id = txt_id \
  WHERE txt_owner_id > 1 \
    AND (txt_title ilike '%delete%' OR txt_ckn ilike '%delete%' OR txt_title IS NULL) \
  ORDER BY txt_owner_id,txt_id;"
myRQC.query(queryFindTextToDelete)
dfDelTexts = pd.DataFrame(myRQC.getAllResults(), columns=myRQC.getColumnNames())
toDelTxtIDs = []
if 'txt_id' in dfDelTexts.columns:
  toDelTxtIDs = pd.unique(sorted(list(dfDelTexts['txt_id'])))
if len(toDelTxtIDs) > 0:
  myRSH.saveDataFrame(dfDelTexts,
                      outdir='output', 
                      filename=f"texts_ready_to_delete{dbname}",
                      extType=outputFormat)
  print(f"Marking for delete text ids {toDelTxtIDs}")
#  deleteTexts(toDelTxtIDs)
else:
  print("No texts to process")

In [ ]:
'''
Find active textmetadata of marked for delete text
'''
queryFindTextMetadataToDelete = \
" SELECT tmd_id, txt_id, tmd_owner_id, txt_owner_id \
  FROM textmetadata \
   LEFT JOIN text on tmd_text_id = txt_id \
  WHERE txt_owner_id = 1;"
myRQC.query(queryFindTextMetadataToDelete)
dfDelTextMetadatas = pd.DataFrame(myRQC.getAllResults(), columns=myRQC.getColumnNames())
toDelTmdIDs = []
if 'tmd_id' in dfDelTextMetadatas.columns:
  toDelTmdIDs = pd.unique(sorted(list(dfDelTextMetadatas['tmd_id'])))
if len(toDelTmdIDs) > 0:
  myRSH.saveDataFrame(dfDelTextMetadatas,
                      outdir='output', 
                      filename=f"textmetadatas_ready_to_delete{dbname}",
                      extType=outputFormat)
  print(f"Marking for delete textmetadata ids {toDelTmdIDs}")
#  deleteTextMetadatas(toDelTmdIDs)
else:
  print("No textmetadata to process")

In [ ]:
'''
Find active editions of active text that have 'to delete' in title or of inactive text
'''
queryFindEditionToDelete = \
" SELECT txt_id, txt_owner_id, txt_title, txt_ckn, edn_id, edn_owner_id, edn_description \
  FROM text \
    LEFT JOIN edition ON edn_text_id = txt_id \
  WHERE edn_id IS NOT NULL AND edn_owner_id > 1 AND (edn_description ilike '%delete%' OR txt_owner_id = 1) \
  ORDER BY txt_id, txt_owner_id, edn_owner_id, edn_id;"
myRQC.query(queryFindEditionToDelete)
dfDelEditions = pd.DataFrame(myRQC.getAllResults(), columns=myRQC.getColumnNames())
toDelEdnIDs = []
if 'edn_id' in dfDelEditions.columns:
  toDelEdnIDs = pd.unique(sorted(list(dfDelEditions['edn_id'])))
if len(toDelEdnIDs) > 0:
  myRSH.saveDataFrame(dfDelEditions,
                      outdir='output', 
                      filename=f"editions_ready_to_delete{dbname}",
                      extType=outputFormat)
  print(f"Marking for delete edition ids {toDelEdnIDs}")
#  deleteEditions(toDelEdnIDs)
else:
  print("No editions to process")

In [ ]:

'''
 find all 'TextPhysical' of 'marked for delete' editions not attached to an active edition
'''
textPhysSeqTermID = myRQC.getTermIDStrict('textphysical','sequencetype')

queryFindTextPhysSequenceToDelete = f"\
SELECT tp.seq_id as seqid_del, tp.seq_owner_id as del_owner, tp.seq_label as tplabel,\
  d.edn_description, d.edn_id, d.edn_owner_id, d.edn_sequence_ids \
FROM edition a \
  LEFT JOIN edition d ON a.edn_text_id = d.edn_text_id \
  LEFT JOIN sequence tp ON tp.seq_id = ANY(d.edn_sequence_ids) \
WHERE a.edn_id != d.edn_id AND a.edn_owner_id != 1 AND not a.edn_description ilike '%to delete%' \
  AND d.edn_owner_id = 1 \
  AND not tp.seq_id = ANY(a.edn_sequence_ids) AND tp.seq_type_id = {textPhysSeqTermID} \
ORDER BY tp.seq_id;\
"
myRQC.query(queryFindTextPhysSequenceToDelete)
dfTextPhysicalSequences = pd.DataFrame(myRQC.getAllResults(), columns=myRQC.getColumnNames())
arrToDel = dfTextPhysicalSequences.index[dfTextPhysicalSequences['del_owner'].ne(1)]
toDelSeqIDs = []
if 'seqid_del' in dfTextPhysicalSequences.columns:
  toDelSeqIDs = pd.unique(sorted(list(dfTextPhysicalSequences['seqid_del'].loc[arrToDel])))
arrPrevDel = dfTextPhysicalSequences.index[dfTextPhysicalSequences['del_owner'].eq(1)]
prevDelSeqIDs = pd.unique(sorted(list(dfTextPhysicalSequences['seqid_del'].loc[arrPrevDel])))
print("Deleted TP Sequences:",prevDelSeqIDs)
# delete
if len(toDelSeqIDs) > 0:
  myRSH.saveDataFrame(dfTextPhysicalSequences,
                      outdir='output', 
                      filename=f"textphysical_sequences_to_delete{dbname}",
                      extType=outputFormat)
  print("TP Sequences to delete:",toDelSeqIDs)
  print(f"Marking for delete tp seqids {toDelSeqIDs}")
#  deleteSequences(toDelSeqIDs)
else:
  print("No text physical sequences to process")

In [ ]:

'''
 find all 'Text' sequences of 'marked for delete' editions not attached to an active edition
'''
textSeqTermID = myRQC.getTermIDStrict('text','sequencetype')

queryFindTextSequenceToDelete = f"\
SELECT tx.seq_id as seqid_del, tx.seq_owner_id as del_owner, tx.seq_label as txlabel,\
  d.edn_description, d.edn_id, d.edn_owner_id, d.edn_sequence_ids \
FROM edition a \
  LEFT JOIN edition d ON a.edn_text_id = d.edn_text_id \
  LEFT JOIN sequence tx ON tx.seq_id = ANY(d.edn_sequence_ids) \
WHERE a.edn_id != d.edn_id AND a.edn_owner_id != 1 AND not a.edn_description ilike '%to delete%' \
  AND d.edn_owner_id = 1 \
  AND not tx.seq_id = ANY(a.edn_sequence_ids) AND tx.seq_type_id = {textSeqTermID} \
ORDER BY tx.seq_id;\
"
myRQC.query(queryFindTextSequenceToDelete)
dfTextSequences = pd.DataFrame(myRQC.getAllResults(), columns=myRQC.getColumnNames())
arrToDel = dfTextSequences.index[dfTextSequences['del_owner'].ne(1)]
toDelSeqIDs = []
if 'seqid_del' in dfTextSequences.columns:
  toDelSeqIDs = pd.unique(sorted(list(dfTextSequences['seqid_del'].loc[arrToDel])))
arrPrevDel = dfTextSequences.index[dfTextSequences['del_owner'].eq(1)]
prevDelSeqIDs = pd.unique(sorted(list(dfTextSequences['seqid_del'].loc[arrPrevDel])))
print("Deleted TX Sequences:",prevDelSeqIDs)
# delete
if len(toDelSeqIDs) > 0:
  myRSH.saveDataFrame(dfTextSequences,
                      outdir='output', 
                      filename=f"text_sequences_to_delete{dbname}",
                      extType=outputFormat)
  print("TX Sequences to delete:",toDelSeqIDs)
  print(f"Marking {len(toDelSeqIDs)} for delete tx seqids {toDelSeqIDs}")
#  deleteSequences(toDelSeqIDs)
else:
  print("No text sequences to process")

In [ ]:

'''
find all active 'TextDivision' sequences to be delete:
  active 'TextDivision' sequences attached to text sequence marked for delete
  not attached to an active text sequence of an edition of the same text
'''
textSeqTermID = myRQC.getTermIDStrict('text','sequencetype')
textDivSeqTermID = myRQC.getTermIDStrict('textdivision','text')

queryFindTextDivSequenceToDelete = f"\
SELECT de.edn_id, de.edn_owner_id, de.edn_description as deledn_label, \
  txd.seq_id as txseqid_del, txd.seq_owner_id as txdel_owner, txd.seq_label as txdel_label,\
  tdd.seq_id as tdseqid_del, tdd.seq_owner_id as tddel_owner, tdd.seq_label as tddel_label,\
  txa.seq_id as txseqid_keep, txa.seq_label as tpkeep_label,\
  ae.edn_description as actedn_label \
FROM edition de \
  LEFT JOIN edition ae on ae.edn_text_id = de.edn_text_id \
  LEFT JOIN sequence txd on txd.seq_id = ANY(de.edn_sequence_ids) \
  LEFT JOIN sequence txa on txa.seq_id = ANY(ae.edn_sequence_ids) \
  LEFT JOIN sequence tdd on CONCAT('seq:', tdd.seq_id) = ANY(txd.seq_entity_ids) \
WHERE de.edn_id != ae.edn_id AND de.edn_owner_id = 1 AND ae.edn_owner_id !=1 \
  AND txd.seq_id != txa.seq_id AND txd.seq_type_id = {textSeqTermID} AND txd.seq_owner_id = 1 \
  AND tdd.seq_type_id = {textDivSeqTermID} AND NOT CONCAT('seq:', tdd.seq_id) = ANY(txa.seq_entity_ids) \
  AND txa.seq_type_id = {textSeqTermID} AND txa.seq_owner_id != 1 \
ORDER BY tdd.seq_id; \
"
myRQC.query(queryFindTextDivSequenceToDelete)
dfTextDivisionSequences = pd.DataFrame(myRQC.getAllResults(), columns=myRQC.getColumnNames())
arrToDel = dfTextDivisionSequences.index[dfTextDivisionSequences['tddel_owner'].ne(1)]
arrPrevDel = dfTextDivisionSequences.index[dfTextDivisionSequences['tddel_owner'].eq(1)]
toDelSeqIDs = []
prevDelSeqIDs = []
if 'tdseqid_del' in dfTextDivisionSequences.columns:
  toDelSeqIDs = pd.unique(sorted(list(dfTextDivisionSequences['tdseqid_del'].loc[arrToDel])))
  prevDelSeqIDs = pd.unique(sorted(list(dfTextDivisionSequences['tdseqid_del'].loc[arrPrevDel])))
print("TD Sequences to delete:",toDelSeqIDs)
print("Deleted TD Sequences:",prevDelSeqIDs)
# delete
if len(toDelSeqIDs) > 0:
  myRSH.saveDataFrame(dfTextDivisionSequences,
                      outdir='output', 
                      filename=f"textdivision_sequences_to_delete{dbname}",
                      extType=outputFormat)
  print(f"Marking {len(toDelSeqIDs)} for delete td seqids {toDelSeqIDs}")
#  deleteSequences(toDelSeqIDs)
else:
  print("No text division sequences to process")

In [ ]:
'''
find all active token to be delete:
  active token not attached to text division sequence not marked for delete
'''
textDivSeqTermID = myRQC.getTermIDStrict('textdivision','text')

queryFindTextDivSequenceToDelete = f" \
  SELECT td \
  FROM token \
  WHERE tok_id NOT IN (\
    SELECT DISTINCT(REPLACE(UNNEST(seq_entity_ids),'tok:',''))::int AS tokID \
    FROM sequence \
    WHERE seq_type_id = {textDivSeqTermID} AND seq_owner_id != 1 \
  ) \
  ORDER BY tok_id;"




In [ ]:

'''
  find all active 'LinePhysical' sequences to be delete:
  active 'LinePhysical' sequences attached to textphysical sequence marked for delete
  not attached to an active textphysical sequence of an edition of the same text
'''
textPhysSeqTermID = myRQC.getTermIDStrict('textphysical','sequencetype')
linePhysSeqTermID = myRQC.getTermIDStrict('linephysical','textphysical')

queryFindLinePhysSequenceToDelete = f"\
SELECT de.edn_id, de.edn_owner_id, de.edn_description as deledn_label, \
  tpd.seq_id as tpseqid_del, tpd.seq_owner_id as tpdel_owner, tpd.seq_label as tpdel_label,\
  lpd.seq_id as lpseqid_del, lpd.seq_owner_id as lpdel_owner, lpd.seq_label as lpdel_label,\
  tpa.seq_id as tpseqid_keep, tpa.seq_label as tpkeep_label,\
  ae.edn_description as actedn_label \
FROM edition de \
  LEFT JOIN edition ae on ae.edn_text_id = de.edn_text_id \
  LEFT JOIN sequence tpd on tpd.seq_id = ANY(de.edn_sequence_ids) \
  LEFT JOIN sequence tpa on tpa.seq_id = ANY(ae.edn_sequence_ids) \
  LEFT JOIN sequence lpd on CONCAT('seq:', lpd.seq_id) = ANY(tpd.seq_entity_ids) \
WHERE de.edn_id != ae.edn_id AND de.edn_owner_id = 1 AND ae.edn_owner_id !=1 \
  AND tpd.seq_id != tpa.seq_id AND tpd.seq_type_id = {textPhysSeqTermID} AND tpd.seq_owner_id = 1 \
  AND lpd.seq_type_id = {linePhysSeqTermID} AND NOT CONCAT('seq:', lpd.seq_id) = ANY(tpa.seq_entity_ids) \
  AND tpa.seq_type_id = {textPhysSeqTermID} AND tpa.seq_owner_id != 1 \
ORDER BY lpd.seq_id; \
"
myRQC.query(queryFindLinePhysSequenceToDelete)
dfLinePhysicalSequences = pd.DataFrame(myRQC.getAllResults(), columns=myRQC.getColumnNames())
arrToDel = dfLinePhysicalSequences.index[dfLinePhysicalSequences['lpdel_owner'].ne(1)]
arrPrevDel = dfLinePhysicalSequences.index[dfLinePhysicalSequences['lpdel_owner'].eq(1)]
toDelSeqIDs = []
prevDelSeqIDs = []
if 'lpseqid_del' in dfLinePhysicalSequences.columns:
  toDelSeqIDs = pd.unique(sorted(list(dfLinePhysicalSequences['lpseqid_del'].loc[arrToDel])))
  prevDelSeqIDs = pd.unique(sorted(list(dfLinePhysicalSequences['lpseqid_del'].loc[arrPrevDel])))
print("LP Sequences to delete:",toDelSeqIDs)
print("Deleted LP Sequences:",prevDelSeqIDs)
# delete
if len(toDelSeqIDs) > 0:
  myRSH.saveDataFrame(dfLinePhysicalSequences,
                      outdir='output', 
                      filename=f"linephysical_sequences_to_delete{dbname}",
                      extType=outputFormat)
  print(f"Marking {len(toDelSeqIDs)} for delete tp seqids {toDelSeqIDs}")
#  deleteSequences(toDelSeqIDs)
else:
  print("No line physical sequences to process")

In [ ]:

'''
Find all to be deleted syllables and ignore any that are in an active Line Physical sequence
'''
textPhysSeqTermID = myRQC.getTermIDStrict('textphysical','sequencetype')
linePhysSeqTermID = myRQC.getTermIDStrict('linephysical','textphysical')

# find all syllables from 'marked for delete' LinePhysical sequence that are not makred for delete
queryFindLinePhysSyllablesToDelete = f"\
SELECT  a.lpseqid_del as lpseqid_del, a.sclgid as sclgid, scl_id, scl_owner_id \
FROM (SELECT UNNEST(lpd.seq_entity_ids) as sclgid, lpd.seq_id as lpseqid_del \
      FROM sequence lpd \
      WHERE lpd.seq_owner_id = 1 AND lpd.seq_type_id = {linePhysSeqTermID} \
      ORDER BY sclgid) a \
  LEFT JOIN syllablecluster ON REPLACE(a.sclgid,'scl:','')::int = scl_id \
WHERE scl_owner_id != 1 \
"
toDelSclIDs = []
toDelSclGIDs = []
cntDelScl = 0
myRQC.query(queryFindLinePhysSyllablesToDelete)
if myRQC.getResultCount() > 0:
  dfSyllablesToDel = pd.DataFrame(myRQC.getAllResults(), columns=myRQC.getColumnNames())
  delLPSeqIDs = pd.unique(sorted(list(dfSyllablesToDel['lpseqid_del'])))
  toDelSclIDs = pd.unique(sorted(list(dfSyllablesToDel['scl_id'])))
  toDelSclGIDs = pd.unique(sorted(list(dfSyllablesToDel['sclgid'])))
  cntDelLPSeq = len(delLPSeqIDs)
  print(f"{cntDelLPSeq} line physical deleted:",delLPSeqIDs)
  cntDelScl = len(toDelSclIDs)
print(f"{cntDelScl} syllables to delete:",toDelSclIDs)


if cntDelScl > 0:
  '''
  Find all active syllables and ignore any that are in marked for delete Line Physical sequence
  '''
  # find all syllables from active LinePhysical sequences that are not marked for delete
  queryFindActiveLinePhysSyllables = f"\
  SELECT  a.lpseqid_act as lpseqid_act, a.sclgid as sclgid, scl_id, scl_owner_id \
  FROM (SELECT UNNEST(lpa.seq_entity_ids) as sclgid, lpa.seq_id as lpseqid_act \
        FROM sequence lpa \
        WHERE lpa.seq_owner_id != 1 AND lpa.seq_type_id = {linePhysSeqTermID} \
        ORDER BY sclgid) a \
    LEFT JOIN syllablecluster ON REPLACE(a.sclgid,'scl:','')::int = scl_id \
  WHERE scl_owner_id != 1 \
  "
  myRQC.query(queryFindActiveLinePhysSyllables)
  if myRQC.getResultCount() > 0:
    dfActiveSyllables = pd.DataFrame(myRQC.getAllResults(), columns=myRQC.getColumnNames())
    actSclIDs = pd.unique(sorted(list(dfActiveSyllables['scl_id'])))
    actSclGIDs = pd.unique(sorted(list(dfActiveSyllables['sclgid'])))
    cntActScl = len(actSclIDs)
    cntActSclGID = len(actSclGIDs)
    print(f"{cntActScl} active syllables:",actSclIDs)
    print(f"{cntActSclGID} GIDs of active syllables:",actSclGIDs)

    # check that the del syllables found are not also in active syllable set from 
    # an active line physical (from cloning)
    toKeepSclGIDsOfDels = [ sclGID for sclGID in toDelSclGIDs if sclGID in actSclGIDs]
    cntToKeepSclIDs = len(toKeepSclGIDsOfDels)
    print(f"{cntToKeepSclIDs} toDelSclGIDs found to be active syllables:",toKeepSclGIDsOfDels)
    if cntToKeepSclIDs > 0:
      #remove active syllables from the to delete syllables
      newToDelSclIDs = [sclID for sclID in toDelSclIDs if 'scl:' + str(sclID) not in toKeepSclGIDsOfDels]
      if len(newToDelSclIDs) + cntToKeepSclIDs != cntDelScl:
        print(f"WARNING: toDelSclGIDs count {cntDelScl} does not equal new count {len(newToDelSclIDs)} plus active count {cntToKeepSclIDs}")
        print(f"aborting to mark syllables for delete {toDelSclIDs}")
        cntDelScl = 0
      else:
        toDelSclIDs = newToDelSclIDs
        # delete
        cntDelScl = len(toDelSclIDs)
  myRSH.saveDataFrame(dfSyllablesToDel,
                      outdir='output', 
                      filename=f"linephysical_syllables_to_delete{dbname}",
                      extType=outputFormat)
  myRSH.saveDataFrame(dfActiveSyllables,
                      outdir='output', 
                      filename=f"linephysical_syllables_to_keep{dbname}",
                      extType=outputFormat)
  print(f"Marking {cntDelScl} syllables for delete; lp sclids {toDelSclIDs}")
#  if cntDelScl > 0:
#    deleteSyllableClusters(toDelSclIDs)
else:
  print("No line physical sequence syllables to process")


In [ ]:

'''
Find and delete all segments attached only to syllables marked for delete
'''
textPhysSeqTermID = myRQC.getTermIDStrict('textphysical','sequencetype')
linePhysSeqTermID = myRQC.getTermIDStrict('linephysical','textphysical')
imgBaselineTermID = myRQC.getTermIDStrict('image','baselinetype')
tagPaleographyTermID = myRQC.getTermIDStrict('paleography','tagtype')

# Find all image segments attached to a syllable marked for delete
queryFindSegmentsOfSyllablesToDelete = f"\
  SELECT seg_id, dsc.scl_id AS scl_del, \
    dsc.scl_owner_id AS del_owner, \
    bln_id, bln_type_id, bln_owner_id, seg_owner_id, seg_image_pos, \
    dsc.scl_grapheme_ids AS grad, dsc.scl_owner_id \
  FROM segment \
  LEFT JOIN syllablecluster dsc ON dsc.scl_segment_id=seg_id \
  LEFT JOIN baseline ON bln_id=ANY(seg_baseline_ids) \
  WHERE dsc.scl_id IS NOT NULL AND dsc.scl_owner_id = 1 \
    AND seg_owner_id !=1 AND seg_image_pos IS NOT NULL \
    AND bln_type_id={imgBaselineTermID} \
  ORDER BY seg_owner_id, seg_id, dsc.scl_id;\
"
delSclSegIDs = []
cntDelSclSegs = 0
myRQC.query(queryFindSegmentsOfSyllablesToDelete)
if myRQC.getResultCount() > 0:
  dfDelSclSegments = pd.DataFrame(myRQC.getAllResults(), columns=myRQC.getColumnNames())
  delSclSegIDs = pd.unique(sorted(list(dfDelSclSegments['seg_id'])))
  myRSH.saveDataFrame(dfDelSclSegments,
                      outdir='output', 
                      filename=f"del_syllables_image_segments{dbname}",
                      extType=outputFormat)
  cntDelSclSegs = len(delSclSegIDs)
  print(f"{cntDelSclSegs} segments attached to deleted encoding clusters:",delSclSegIDs)


if cntDelSclSegs > 0:
  '''
Find all image segments attached only to a syllable marked for delete
  '''
  queryFindImageSegmentsToDelete = f"\
    SELECT seg_id, dsc.scl_id AS scl_del, sca.scl_id AS scl_act,\
        dsc.scl_owner_id AS del_owner, sca.scl_owner_id AS act_owner,\
        bln_id, bln_type_id, bln_owner_id, seg_image_pos,\
        dsc.scl_grapheme_ids AS grad, dsc.scl_owner_id,\
        sca.scl_owner_id, sca.scl_grapheme_ids AS graa \
    FROM segment \
    LEFT JOIN syllablecluster dsc ON dsc.scl_segment_id=seg_id \
    LEFT JOIN baseline ON bln_id=ANY(seg_baseline_ids) \
    LEFT JOIN syllablecluster sca ON sca.scl_segment_id=seg_id \
    LEFT JOIN sequence on CONCAT('seq:', sca.scl_id) = ANY(seq_entity_ids) \
    WHERE dsc.scl_id is not NULL AND dsc.scl_owner_id = 1 AND dsc.scl_id != sca.scl_id AND sca.scl_owner_id != 1 \
      AND seg_owner_id !=1 AND seg_image_pos is not NULL AND seq_owner_id = 1 AND seq_type_id = {linePhysSeqTermID} \
      AND bln_type_id={imgBaselineTermID} \
    ORDER BY seg_id, dsc.scl_id;\
  "
  myRQC.query(queryFindImageSegmentsToDelete)
  if myRQC.getResultCount() > 0:
    dfDelImageSegments = pd.DataFrame(myRQC.getAllResults(), columns=myRQC.getColumnNames())
    toDelSegIDs = pd.unique(sorted(list(dfDelImageSegments['seg_id'])))
    cntDelSegs = len(toDelSegIDs)
    print(f"{cntDelSegs} toDelSegIDs segments to delete:",toDelSegIDs)
    myRSH.saveDataFrame(dfDelImageSegments,
                        outdir='output', 
                        filename=f"del_syllable_segments_to_delete{dbname}",
                        extType=outputFormat)
    print(f"Marking {cntDelSegs} syllables for delete; lp sclids {toDelSegIDs}")
#    deleteSegments(toDelSegIDs)
  else:
    print("No image segments to delete found")
else:
  print("No image segments to process")




In [ ]:

'''
Find all syllableclusters that are 'marked for delete' that are tagged
'''
tagPaleographyTermID = myRQC.getTermIDStrict('paleography','tagtype')

# Find all annotation with links to a syllable marked for delete
queryFindAnnotationsWithSyllablesToRemove = f"\
  SELECT ano_id, ano_type_id as tagID, ano_linkto_ids as links, \
         ARRAY_AGG(CONCAT('scl:',a.scl_id)) AS sclLinksToRemove \
  FROM annotation \
    LEFT JOIN \
      (SELECT scl_id \
      FROM syllablecluster \
      WHERE scl_owner_id = 1 \
        AND scl_id IN \
        (SELECT  DISTINCT(REPLACE(UNNEST(ano_linkto_ids),'scl:',''))::int as scl_id \
         FROM annotation \
         WHERE ano_owner_id != 1 \
           AND ano_type_id IN (SELECT trm_id \
                               FROM term \
                               WHERE trm_parent_id in (SELECT trm_id \
                                                       FROM term \
                                                       WHERE trm_parent_id = {tagPaleographyTermID}))\
           AND ano_linkto_ids[1] LIKE 'scl%' \
         ORDER BY scl_id) \
      ORDER BY scl_id) a ON CONCAT('scl:',a.scl_id) = ANY(ano_linkto_ids) \
  WHERE a.scl_id IS NOT null \
  GROUP BY ano_id; \
"
myRQC.query(queryFindAnnotationsWithSyllablesToRemove)
if myRQC.getResultCount() > 0:
  dfAnoWithDelScls = pd.DataFrame(myRQC.getAllResults(), columns=myRQC.getColumnNames())
  myRSH.saveDataFrame(dfAnoWithDelScls,
                      outdir='output', 
                      filename=f"anoTags_linked_with_del_syllables{dbname}",
                      extType=outputFormat)
  anoInfoWithDelScls = myRQC.getRowsAsIndexMultiDict(columnName = 'ano_id')
  for anoID in anoInfoWithDelScls.keys():
    anoInfo = anoInfoWithDelScls[anoID][0]
    tag = myRQC.getTermCode(anoInfo['tagid'])
    links = anoInfo['links']
    sclLinksToRemove = anoInfo['scllinkstoremove']
    print(f" tag {tag} in ano {anoID} has {len(links)} links with {len(sclLinksToRemove)} to be removed")
    for sclGID in sclLinksToRemove:
      links.remove(sclGID)
    print(f"tag {tag} now has {len(links)} links for updating anoID {anoID}")
#    myRQC.update("annotation",{"ano_linkto_ids":"{"+','.join([str(i) for i in links])+"}"},{"ano_id":anoID})
#    if myRQC.hasError():
#      print(f"Error while updating anoID {anoID} - {myRQC.getError()}")
#    else:
#      print(f"Update of tag {tag} (ano:{anoID}) was successful")
else:
  print("No sclIDs linked to annoTag to be removed")





In [ ]:
myRQC.query(
  '''
   WITH allLinkedTextDivSeqIDs AS (-- list all TD sequences in an active text sequence
    SELECT replace(unnest(seq_entity_ids),'seq:','')::int as seqID
    FROM sequence
    WHERE seq_type_id = 738 AND seq_owner_id != 1
   ),
   activeTextDivIDs AS (-- seq ids of active text division sequence linked to text sequence
    SELECT seq_id
    FROM sequence
    WHERE seq_owner_id != 1
     AND seq_id IN (SELECT seqID FROM allLinkedTextDivSeqIDs)
    ORDER BY seq_id
   ),
   orphTextDivIDs AS (-- seq ids of text division sequence not linked to an active text sequence
    SELECT seq_id
    FROM sequence
    WHERE seq_type_id = 739 AND seq_owner_id != 1
     AND NOT seq_id IN (SELECT seqID FROM allLinkedTextDivSeqIDs)
   ),
   orphWordGIDs AS (
    SELECT seq_id AS tdseq_id, UNNEST(seq_entity_ids) AS orph_gid
    FROM sequence
    WHERE seq_id IN (SELECT seq_id FROM orphTextDivIDs)
   ),
   wordGIDs AS (
    SELECT seq_id AS tdseq_id, UNNEST(seq_entity_ids) AS word_gid
    FROM sequence
    WHERE seq_id IN ( SELECT seq_id FROM activeTextDivIDs)
   ),
   orphTokGIDs AS (
    SELECT *
    FROM orphWordGIDs
    WHERE orph_gid LIKE 'tok:%'
    ORDER BY tdseq_id, orph_gid
   ),
   orphTok_n_IDs AS (
    SELECT 'tok' AS prefix, REPLACE(orph_gid, 'tok:', '')::int AS entid
    FROM orphTokGIDs
   ),
   orphTokIDs AS (
    SELECT DISTINCT(entid) AS tok_id
    FROM orphTok_n_IDs
    ORDER BY entid
   ),
   orphCmpGIDs AS (
    SELECT *
    FROM orphWordGIDs
    WHERE orph_gid like 'cmp:%'
    ORDER BY orph_gid
   ),
   orphCmp_n_IDs AS (
    SELECT 'cmp' AS prefix, REPLACE(orph_gid, 'cmp:', '')::int AS entid
    FROM orphCmpGIDs
   ),
   orphCmpIDs AS (
    SELECT DISTINCT(entid) AS cmp_id
    FROM orphCmp_n_IDs
    ORDER BY entid
   ),
   orphPrefixEntIDs AS ( -- all word prefix, id in orphaned text divisions
    SELECT * 
    FROM (
     SELECT * FROM orphCmp_n_IDs
     UNION
     SELECT * FROM orphTok_n_IDs
     ) a
    ORDER BY a.prefix, a.entid
   ),
   misNeedCleanSeqIDs AS ( --find all sequence containing orphaned words needing to be cleaned
    SELECT DISTINCT(seq_id), ARRAY_AGG(orph_gid)::text removeGIDs
    FROM sequence
     LEFT JOIN orphWordGIDs ON orph_gid = ANY(seq_entity_ids)
    WHERE seq_type_id != 739 AND tdseq_id IS NOT null
    GROUP BY seq_id
    ORDER BY seq_id
   ),
   adoptedWordGIDs AS ( --find all ent gids in another text division
    SELECT seq_id, tdseq_id, orph_gid
    FROM sequence
     LEFT JOIN orphWordGIDs ON orph_gid = ANY(seq_entity_ids)
    WHERE seq_type_id = 739 AND tdseq_id IS NOT null
     AND seq_id IN (SELECT seq_id FROM activeTextDivIDs)
    ORDER BY tdseq_id, orph_gid
   ),
   cmpContainedOrphGIDs as ( --find all compounds containing orphaned word GIDs
    SELECT cmp_id, orph_gid
    FROM compound
     LEFT JOIN orphWordGIDs ON orph_gid = ANY(cmp_component_ids)
    WHERE orph_gid IS NOT null
    ORDER BY cmp_id, orph_gid
   )

   SELECT a.prop,a.val 
   FROM (
     SELECT 'activeTDCount' AS prop, COUNT(seq_id)::text AS val FROM activeTextDivIDs
     UNION
     SELECT 'activeTDSeqIDs' AS prop, ARRAY_AGG(seq_id)::text AS val FROM activeTextDivIDs
     UNION
     SELECT 'orphTDCount' AS prop, COUNT(seq_id)::text AS val FROM orphTextDivIDs
     UNION
     SELECT 'orphTDSeqIDs' AS prop, ARRAY_AGG(seq_id)::text AS val FROM orphTextDivIDs
     UNION
     SELECT 'orphWordCount' AS prop, COUNT(DISTINCT(orph_gid))::text AS val FROM orphWordGIDs
     UNION
     SELECT 'orphWordGIDs' AS prop, ARRAY_AGG(DISTINCT(orph_gid))::text AS val FROM orphWordGIDs
     UNION
     SELECT 'adoptedWordCount' AS prop, COUNT(orph_gid)::text AS val FROM adoptedWordGIDs
     UNION
     SELECT 'adoptedWordGIDs' AS prop, ARRAY_AGG(orph_gid)::text AS val FROM adoptedWordGIDs
     UNION
     SELECT 'cmpContainedWordCount' AS prop, COUNT(orph_gid)::text AS val FROM cmpContainedOrphGIDs
     UNION
     SELECT 'cmpContainedWordGIDs' AS prop, ARRAY_AGG(orph_gid)::text AS val FROM cmpContainedOrphGIDs
     UNION
     SELECT 'orphTokCount' AS prop, COUNT(tok_id)::text AS val FROM orphTokIDs
     UNION
     SELECT 'orphTokIDs' AS prop, ARRAY_AGG(tok_id)::text AS val FROM orphTokIDs
     UNION
     SELECT 'orphCmpCount' AS prop, COUNT(cmp_id)::text AS val FROM orphCmpIDs
     UNION
     SELECT 'orphCmpIDs' AS prop, ARRAY_AGG(cmp_id)::text AS val FROM orphCmpIDs
     UNION
     SELECT CONCAT('removeFromSeq:',seq_id) AS prop, removeGIDs AS val FROM misNeedCleanSeqIDs
      ) a
   ORDER BY a.prop
  '''
  )
if myRQC.getResultCount() > 0:
  dfTextDivWordCleanupInfo = pd.DataFrame(myRQC.getAllResults(), columns=myRQC.getColumnNames())
  myRSH.saveDataFrame(dfTextDivWordCleanupInfo,
                      outdir='output', 
                      filename=f"Text Div and Word Cleanup Info {dbname}",
                      extType=outputFormat)
  dictInfo = myRQC.getRowsAsKVDict(kColIndex = 0, vColIndex =1)
  gidStillActiveWords = None
  cntStillActiveWords = int(dictInfo['adoptedWordCount'])
  if cntStillActiveWords > 0:
    gidStillActiveWords = dictInfo['adoptedWordGIDs'].strip('{').strip('}').split(',').sort()
  gidContainedActiveWords = None
  cntContainedActiveWords = int(dictInfo['cmpContainedWordCount'])
  if cntContainedActiveWords > 0:
    gidContainedActiveWords = dictInfo['cmpContainedWordGIDs'].strip('{').strip('}').split(',').sort()
  orphTDSeqIDs = None
  cntOrphTDSeqIDs = int(dictInfo['orphTDCount'])
  if cntOrphTDSeqIDs > 0:
    orphTDSeqIDs = [int(x) for x in dictInfo['orphTDSeqIDs'].strip('{').strip('}').split(',')]
#    deleteSequences(orphTDSeqIDs)
  orphTokIDs = None
  cntOrphTokIDs = int(dictInfo['orphTokCount'])
  if cntOrphTokIDs > 0:
    orphTokIDs = [int(x) for x in dictInfo['orphTokIDs'].strip('{').strip('}').split(',')]
#    deleteTokens(orphTokIDs)
  orphCmpIDs = None
  cntOrphCmpIDs = int(dictInfo['orphCmpCount'])
  if cntOrphCmpIDs > 0:
    orphCmpIDs = [int(x) for x in dictInfo['orphCmpIDs'].strip('{').strip('}').split(',')]
  if str(dictInfo.keys()).find('removeFromSeq') > -1:
    cleanUpSeqCmdList = [['seq:'+key[14:],'remove',dictInfo[key].strip('{').strip('}').split(',')] for key in dictInfo.keys() if key.startswith('removeFromSeq')]
    adjustlist = [['all',dbname,cleanUpSeqCmdList]]
#    rlm.adjustChildLinks(adjustlist,myRQC)

elif myRQC.hasError():
  print(f"Fail to obtain orphan info {myRQC.getError()}")


In [ ]:
#rlm.adjustChildLinks(adjustlist,myRQC)
#deleteSequences(orphTDSeqIDs)
#orphTDSeqIDs
#deleteTokens(orphTokIDs)
#orphTokIDs
#adjustlist